# xG

In [44]:
! pip install gandula


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Esse notebook mostra como implementar um modelo de Expectativa de Gols do zero até ter ele completo.

## Imports

In [45]:
import gandula

In [46]:
gandula

<module 'gandula' from 'c:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\3. Workshop FAME - xG\\0. Competição\\famexG\\Lib\\site-packages\\gandula\\__init__.py'>

In [47]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

## Coleta de dados

A PFF fornece dois tipos de dado, que podem vir de jeitos diferentes:

- dados de evento: json ou via API
- dados de tracking: json

O dado de todos os chutes de todas as partidas do dataset foi pre-processado para facilitar nossas vida. Em outros notebooks da biblioteca, você pode checar como ele foi gerado. Podemos extraí-lo da seguinte forma:

In [48]:
shot_frames = gandula.loader.read_pickle(
    'C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\3. Workshop FAME - xG\\0. Competição\\enhaced_frames_shots.pkl'
)

## Exploração dos dados

Vamos primeiro ver como os chutes aparecem.

In [49]:
type(shot_frames)

list

Se é uma lista vamos ver com os chutes aparecem.

In [50]:
shot_frames[:5]

[{'frame': Frame 9282 - 130.66371,
  'event': PFF_PossessionEventType.SHOT=6511176},
 {'frame': Frame 20398 - 501.567947,
  'event': PFF_PossessionEventType.SHOT=6512525},
 {'frame': Frame 22485 - 571.20425,
  'event': PFF_PossessionEventType.SHOT=6511957},
 {'frame': Frame 36063 - 1024.257303,
  'event': PFF_PossessionEventType.SHOT=6511049},
 {'frame': Frame 41923 - 1219.786165,
  'event': PFF_PossessionEventType.SHOT=6511298}]

## Tratamento dos dados

No modelo de xG que vamos desenvolver aqui, vamos precisar extrair o ângulo para o gol e a distância. Além disso, é necessário saber se o chute resultou em gol ou não. Vamos gerar esses dados para cada chute.

Para facilitar os cálculos, vou transformar os dados gerados pelo gandula em um dataframe.

In [51]:
data = [
    {
        'event_id': frame['frame'].event_id,
        'frame_id': frame['frame'].frame_id,
        'x': frame['event'].shootingEvent.shotPointX,
        'y': frame['event'].shootingEvent.shotPointY,
        'outcome': frame['event'].shootingEvent.shotOutcomeType,
        'body_part': frame['event'].shootingEvent.shotBodyType,
        'set_piece': frame['frame'].event.setpiece_type,
        'initial_height_type':frame['event'].shootingEvent.shotInitialHeightType,
        'pressureType': frame['event'].shootingEvent.pressureType,
        'shotNatureType': frame['event'].shootingEvent.shotNatureType
    }
    for frame in shot_frames
]

df = pd.DataFrame(data)
df = (
    df.drop_duplicates()
)  # TODO: fix enhanced shot frames: there shouldn't be duplicates
df.head()

,event_id,frame_id,x,y,outcome,body_part,set_piece,initial_height_type,pressureType,shotNatureType
0,6629878,9282,-40.281,-4.868,ShotOutcomeType.ON_TARGET,BodyType.LEFT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT
1,6631191,20398,20.974,-12.711,ShotOutcomeType.OFF_TARGET,BodyType.RIGHT_FOOT,None,ShotHeightType.SHORT,PressureType.PRESSURED,ShotNatureType.POWER
2,6630642,22485,16.634,-13.580,ShotOutcomeType.GOAL,BodyType.RIGHT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT
3,6629751,36063,15.746,14.719,ShotOutcomeType.OFF_TARGET,BodyType.LEFT_FOOT,None,ShotHeightType.OVER,None,ShotNatureType.PLACEMENT
4,6630005,41923,-4.834,-21.644,ShotOutcomeType.OFF_TARGET,BodyType.RIGHT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT


In [52]:
df.shape

(1518, 10)

Para extrair a distância, vamos utilizar a distância euclideana. Para fazer o cálculo, precisamos considerar que os chutes podem sair dos dois lados do campo. A PFF tem como padrão nos seus dados direcionar o campo de acordo com o que a câmera mostra na transmissão. Além disso, as coordenadas são centralizadas no meio de campo. Vamos assumir que os atacantes estão sempre no campo adversário para chutar. Com isso, conseguimos extrair corretamente o dado.

Iremos setar como padrão que o time que está chutando ataca sempre da direita para a esquerda, para facilitar o cálculo. Isso quer dizer que o centro do gol está sempre na coordenada y = 0 e x = 52.5.

In [53]:
# espelhando chutes com x negativo
df.loc[df['x'] < 0, 'y'] = df.loc[df['x'] < 0, 'y'] * -1
df.loc[df['x'] < 0, 'x'] = df.loc[df['x'] < 0, 'x'] * -1

# calculando a distância do chute para o gol
df['dist'] = np.sqrt((df['x'] - 52.5) ** 2 + (df['y'] - 0) ** 2)

df.head()

,event_id,frame_id,x,y,outcome,body_part,set_piece,initial_height_type,pressureType,shotNatureType,dist
0,6629878,9282,40.281,4.868,ShotOutcomeType.ON_TARGET,BodyType.LEFT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT,13.152999
1,6631191,20398,20.974,-12.711,ShotOutcomeType.OFF_TARGET,BodyType.RIGHT_FOOT,None,ShotHeightType.SHORT,PressureType.PRESSURED,ShotNatureType.POWER,33.992031
2,6630642,22485,16.634,-13.580,ShotOutcomeType.GOAL,BodyType.RIGHT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT,38.350833
3,6629751,36063,15.746,14.719,ShotOutcomeType.OFF_TARGET,BodyType.LEFT_FOOT,None,ShotHeightType.OVER,None,ShotNatureType.PLACEMENT,39.591735
4,6630005,41923,4.834,21.644,ShotOutcomeType.OFF_TARGET,BodyType.RIGHT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT,52.349883


Para extrair o ângulo, precisamos pensar na geometria do chute.
Para calcular o ângulo do gol, θ, tomamos a coordenada (x,y) do chute
- x é a distância ao longo da borda do campo desde a linha do gol;
- y é a distância do meio do campo.
- (0,0) é o ponto na linha de fundo no meio do gol.
O ângulo entre dois vetores que apontam para as traves é:
$$
\tan(\theta) = \frac{7.32x}{x^2 + y^2 - \left(\frac{7.32}{2}\right)^2}
$$
- onde 7.32 é o tamanho do gol.


In [54]:
df['ang'] = np.arctan((7.32 * df['x']) / (df['x'] ** 2 + (7.32 - df['y']) ** 2))

df.head()

,event_id,frame_id,x,y,outcome,body_part,set_piece,initial_height_type,pressureType,shotNatureType,dist,ang
0,6629878,9282,40.281,4.868,ShotOutcomeType.ON_TARGET,BodyType.LEFT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT,13.152999,0.179112
1,6631191,20398,20.974,-12.711,ShotOutcomeType.OFF_TARGET,BodyType.RIGHT_FOOT,None,ShotHeightType.SHORT,PressureType.PRESSURED,ShotNatureType.POWER,33.992031,0.180536
2,6630642,22485,16.634,-13.580,ShotOutcomeType.GOAL,BodyType.RIGHT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT,38.350833,0.169025
3,6629751,36063,15.746,14.719,ShotOutcomeType.OFF_TARGET,BodyType.LEFT_FOOT,None,ShotHeightType.OVER,None,ShotNatureType.PLACEMENT,39.591735,0.363845
4,6630005,41923,4.834,21.644,ShotOutcomeType.OFF_TARGET,BodyType.RIGHT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT,52.349883,0.153607


Por fim, vamos extrai quando temos um gol e quando não.

In [55]:
df['gol'] = df['outcome'] == gandula.providers.pff.schema.event.ShotOutcomeType.GOAL
df.head()

,event_id,frame_id,x,y,outcome,body_part,set_piece,initial_height_type,pressureType,shotNatureType,dist,ang,gol
0,6629878,9282,40.281,4.868,ShotOutcomeType.ON_TARGET,BodyType.LEFT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT,13.152999,0.179112,False
1,6631191,20398,20.974,-12.711,ShotOutcomeType.OFF_TARGET,BodyType.RIGHT_FOOT,None,ShotHeightType.SHORT,PressureType.PRESSURED,ShotNatureType.POWER,33.992031,0.180536,False
2,6630642,22485,16.634,-13.580,ShotOutcomeType.GOAL,BodyType.RIGHT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT,38.350833,0.169025,True
3,6629751,36063,15.746,14.719,ShotOutcomeType.OFF_TARGET,BodyType.LEFT_FOOT,None,ShotHeightType.OVER,None,ShotNatureType.PLACEMENT,39.591735,0.363845,False
4,6630005,41923,4.834,21.644,ShotOutcomeType.OFF_TARGET,BodyType.RIGHT_FOOT,None,ShotHeightType.BOTTOM_THIRD,None,ShotNatureType.PLACEMENT,52.349883,0.153607,False


Além do ângulo, é importante temos mais insighs sobre comos chutes são feitos e sua distribuição

In [56]:
df['body_part'].value_counts()

body_part
BodyType.RIGHT_FOOT         768
BodyType.LEFT_FOOT          466
BodyType.HEAD               261
BodyType.RIGHT_BACK_HEEL      4
BodyType.LEFT_KNEE            4
BodyType.RIGHT_SHIN           4
BodyType.LEFT_SHOULDER        3
BodyType.LEFT_SHIN            2
BodyType.RIGHT_KNEE           2
BodyType.RIGHT_THIGH          2
BodyType.RIGHT_HAND           1
BodyType.TWO_HAND_PALM        1
Name: count, dtype: int64

Antes de continuar, vamos limpar um pouco o dado. A natureza de cabeceios é um pouco diferente de chutes. Vamos buscar analisar apenas chutes.

In [59]:
shots = df[
    df['body_part'].isin(
        [
            gandula.providers.pff.schema.event.BodyType.RIGHT_FOOT,
            gandula.providers.pff.schema.event.BodyType.LEFT_FOOT,
            gandula.providers.pff.schema.event.BodyType.RIGHT_KNEE,
            gandula.providers.pff.schema.event.BodyType.LEFT_KNEE,
            gandula.providers.pff.schema.event.BodyType.RIGHT_BACK_HEEL,
            gandula.providers.pff.schema.event.BodyType.LEFT_BACK_HEEL,
            gandula.providers.pff.schema.event.BodyType.RIGHT_THIGH,
            gandula.providers.pff.schema.event.BodyType.LEFT_THIGH,
        ]
    )
]

### DESENVOLVIMENTO E TREINAMENTO DOS MODELOS

In [60]:
# SALVANDO BACKUP DOS DFs PARA NÃO TER QUE RODAR TODAS AS CÉLULAS NOVAMENTE PARA RETORNAR AO DF SEM ALTERAÇÃO

shots_raw = shots.copy()

df_raw = df.copy()

In [61]:
# RESTAURANDO OS DATAFRAMES SEM ALTERAÇÕES

df = df_raw.copy()
shots = shots_raw.copy()

In [62]:
# EXCLUSÃO DA COLUNA OUTCOME (VAZAMENTO DE DADOS) E SET_PIECE (TOTALMENTE NULA)

shots.drop(['outcome','set_piece'], axis=1, inplace=True)

df.drop(['outcome','set_piece'], axis=1, inplace=True)

In [63]:
# LENDO O ARQUIVO DE TESTE E EXTRAINDO A COLUNA DE ID

shotid_test = pd.read_csv('test.csv')
shotid_test = shotid_test[['shot_id']].astype('category')

In [64]:
# CRIANDO UMA COLUNA DE SHOT_ID NO DATASET (USANDO O FORMATO DE SUBMISSÃO - EVENT_ID + FRAME_ID) 

df['shot_id'] = (df['event_id'].astype(str) + '_' + df['frame_id'].astype(str))

df['shot_id'] = df['shot_id'].astype('category')

COMO OS DADOS DO GANDULA POSSUEM TODOS OS REGISTROS (INCLUINDO O DE TESTE), FAREI UM MERGE ENTRE O ARQUIVO COM OS DADOS DE TESTE E O DATASET COMPLETO ATRAVÉS DOS IDs. OS REGISTROS QUE ENCONTRAREM CORRESPONDENTE NO DATASET SÃO OS DE TESTES, OS OUTROS SERÃO USADOS PARA TREINO.

ISSO SERÁ FEITO PARA NÃO CORRER RISCO DE TER VAZAMENTO DE DADOS, OU SEJA, USAR OS DADOS DE TESTE NO TREINAMENTO DO MODELO

In [65]:
# MERGE ENTRE O DATASET DO GANDULA E O DATASET DE TESTE

df_final = pd.merge(df,shotid_test, left_on=df['shot_id'], right_on=shotid_test['shot_id'], how='left', suffixes=('_left','_right'))

# TRANSFORMANDO OS REGISTROS NULOS (QUE NÃO ENCONTRARAM CORRESPONDENTE NO DATASET DE TESTE) COMO 0, O RESTO COMO 1

df_final['shot_id_right'] = np.where(df_final['shot_id_right'].isna(), 0, 1)

# ALTERANDO O NOME DAS COLUNAS 

df_final.rename(columns={'shot_id_right':'test',
                         'shot_id_left':'shot_id'}, inplace=True)

# EXCLUSÃO DE COLUNA DESNECESSÁRIA
df_final.drop('key_0', axis=1, inplace=True)

In [66]:
# FILTRAGEM DAS VARIÁVEIS PREDITORAS
#    EXCLUSÃO DAS COLUNAS REFERENTES AO ID + COLUNA GOL (VARIÁVEL A SER PREDITA) 

X = df_final.drop(['event_id','frame_id','shot_id','gol'], axis=1)

# TRATAMENTO DAS COLUNSA CATEGÓRICAS - TRANSFORMANDO CATEGORIAS EM VARIÁVEIS DUMMIES 

X_dummy = pd.get_dummies(X, prefix="", prefix_sep="")

In [67]:
# ARMAZENAMENTO DOS IDs DO CONJUNTO DE TREINO E TESTE PARA FACILITAR A SUBMISSÃO DA TENTATIVA NO FINAL

ids_test = df_final[df_final['test'] == 1][['event_id','frame_id','shot_id','test']]
ids_train = df_final[df_final['test'] == 0][['event_id','frame_id','shot_id','test']]

# FILTRAGEM DO CONJUNTO DE TREINO E TESTE - VARIÁVEL A SER PREDITA 

y_test = df_final[df_final['test'] == 1]['gol']
y_train = df_final[df_final['test'] == 0]['gol']

# FILTRAGEM DO CONJUNTO DE TREINO E TESTE - VARIÁVEIS PREDITORAS

X_test = X_dummy[X_dummy['test'] == 1].drop('test',axis=1)
X_train = X_dummy[X_dummy['test'] == 0].drop('test',axis=1)

TENTATIVA 3 DE SUBMISSÃO - XGBOOST

In [68]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# INSTANCIANDO O MODELO

xgb = XGBClassifier()

# TREINAMENTO DO MODELO

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=20)

[0]	validation_0-logloss:0.35559	validation_1-logloss:0.35035
[20]	validation_0-logloss:0.17426	validation_1-logloss:0.33957
[40]	validation_0-logloss:0.11580	validation_1-logloss:0.35428
[60]	validation_0-logloss:0.08097	validation_1-logloss:0.37287
[80]	validation_0-logloss:0.06175	validation_1-logloss:0.39778
[99]	validation_0-logloss:0.04806	validation_1-logloss:0.41791


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [69]:
# PREDIÇÃO DO MOELO CRIADO

y_xgb = xgb.predict_proba(X_test)

In [70]:
# AVALIAÇÃO DOS RESULTADOS - ROC AUC 

roc_auc_score(y_test,y_xgb[:,1])

np.float64(0.7220843672456575)

In [71]:
# SALVANDO OS RESULTADOS NO FORMATO DA SUBMISSÃO

tentativa3 = pd.DataFrame({'shot_id': ids_test['shot_id'],
                           'xG': y_xgb[:,1]})

In [72]:
# SALVANDO OS RESULTADOS EM CSV PARA SUBMISSÃO NO KAGGLE

tentativa3.to_csv('tentativa3_xgb.csv', index=False)

TENTATIVA 4 DE SUBMISSÃO - XGBOOST COM `ALGUNS` HIPERPARÂMETROS OTIMIZADOS

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score

# INSTANCIANDO O MODELO
xgb = XGBClassifier(use_label_encoder=False, eval_metric="auc")

# DEFINIÇÃO DE HIPERPARÂMETROS PARA SEREM TESTADOS
param_grid = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# DETERMINAÇÃO DE UM SCORER COM A MÉTRICA DE AVALIAÇÃO DA COMPETIÇÃO
scorer = make_scorer(roc_auc_score, greater_is_better=True, needs_proba=True)

# CONFIGURANDO O OTIMIZADOR
grid_search = GridSearchCV(estimator=xgb, 
                           param_grid=param_grid, 
                           scoring=scorer,
                           cv=5,  # 5-fold cross-validation
                           verbose=3,  # visualização do progresso
                           n_jobs=-1  # uso de todos os núcleos disponíveis para acelerar o processo
                          )

# EXECUTANDO A OTIMIZAÇÃO
grid_search.fit(X_train, y_train)

# EXIBIÇÃO DOS MELHORES HIPERPARÂMETROS ENCONTRADOS
print(f"Melhores Parâmetros: {grid_search.best_params_}")
print(f"Melhor ROC AUC Score: {grid_search.best_score_:.4f}")

In [ ]:
# INSTANCIANDO O MODELO COM OS MELHORES HIPERPARÂMETROS
final_model= XGBClassifier(**grid_search.best_params_)

# TREINAMENTO DO MODELO OTIMIZADO 
final_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
# PREDIÇÃO DO MODELO OTIMIZADO
y_xgb = final_model.predict_proba(X_test)

In [ ]:
# AVALIAÇÃO DOS RESULTADOS - ROC AUC 
roc_auc_score(y_test,y_xgb[:,1])

np.float64(0.7814229814850162)

In [ ]:
# SALVANDO OS RESULTADOS NO FORMATO DA SUBMISSÃO
tentativa4 = pd.DataFrame({'shot_id': ids_test['shot_id'],
                           'xG': y_xgb[:,1]})

In [ ]:
# SALVANDO OS RESULTADOS EM CSV PARA SUBMISSÃO NO KAGGLE
tentativa4.to_csv('tentativa4_xgb.csv', index=False)